<a href="https://colab.research.google.com/github/cooolbabu/GoogleGemini101/blob/testing_claude/AzureDatabricks/Boiler_code_for_Github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Package installs

In [1]:
%pip install anthropic
%pip install PyGithub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 850.5/850.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 350.2/350.2 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 712.4 kB/s eta 0:00:00
  Attempting uninstall: pyjwt
    Found existing installation: PyJWT 2.3.0
    Uninstalling PyJWT-2.3.0:
      Successfully uninstalled PyJWT-2.3.0


### Imports, get keys, get llm client and set model to variable MODEL_NAME

In [2]:
from anthropic import Anthropic
import json
import re
from pprint import pprint
from google.colab import userdata
from github import Github

# Get the OpenAI API key from Colab secrets
github_token=userdata.get('Github_Token')
claude_api_key=userdata.get('Claude_api_key')
# Initialize a GitHub instance
g = Github(github_token)

client = Anthropic(api_key=claude_api_key)
MODEL_NAME = "claude-3-opus-20240229"

### Github helper functions
* read_file_as_string()
* check_in_file(repo_name, file_path, file_content, content_tag, branch)

In [3]:
def read_file_as_string(file_path):
  """
    Reads the file and return a string representation of the file contents

    Parameters:
      file_path (str): Filename including filepath
  """
  try:
      with open(file_path, 'r') as file:
          file_contents = file.read()
      return file_contents
  except FileNotFoundError:
      print(f"File '{file_path}' not found.")
      return None
  except Exception as e:
      print(f"An error occurred: {e}")
      return None

def check_in_file(repo_name, file_path, file_content, content_tag, branch):
  """
    Checks if a specific file exists in a GitHub repository and updates it with new content if it does.
    If the file does not exist, it creates a new file with the provided content.

    This function operates on a specific branch named 'test'. If updating, it will commit the changes with a given content tag as the commit message.
    In case the file needs to be created, it will also use the content tag as the commit message for the new file.

    Parameters:
    - repo_name (str): The name of the repository, formatted as 'username/repository'.
    - file_path (str): The path to the file within the repository. This should include the file name and its extension.
    - file_content (str): The content to be written to the file. This is used both for updating and creating the file.
    - content_tag (str): A message associated with the commit used for updating or creating the file.
    - branch (str): Github branch for the code

    Behavior:
    - If the file exists at the specified path, it updates the file with `file_content`, using `content_tag` as the commit message.
    - If the file does not exist, it creates a new file at the specified path with `file_content`, also using `content_tag` as the commit message for creation.
    - Upon successful update or creation, prints a success message indicating the action taken.
 """

  # Get the repository
  repo = g.get_repo(repo_name)

  try:
      # Get the contents of the file if it exists
      file = repo.get_contents(file_path, ref=branch)

      # Update the file
      repo.update_file(file_path, content_tag, file_content, file.sha, branch=branch)
      print(f"File '{file_path}' updated successfully.")
  except:
      # If the file doesn't exist, create it
      print(f"{file_path}/{file_content} does not exist")
      repo.create_file(file_path, content_tag, file_content, branch=branch)
      print(f"File '{file_path}' created successfully.")

# Setup
1.   System Message
2.   User Message



In [4]:
system_message="You are an AI assistant with a witty sense of humor and a knack for crafting clever puns and wordplay. When a user provides a topic, your task is to generate a list of five puns, play on words, or humorous phrases related to that topic. The wordplay should be original, creative, and aim to elicit a laugh or a groan from the reader."
user_message_content = "fishing"

# Make the call to LLMs

In [5]:
# Create the message with variables
response = client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1000,
    temperature=0,
    system=system_message,
    messages=[
        {"role": "user", "content": user_message_content}
    ]
)

# Assuming you have a client setup for interaction. Ensure to configure your OpenAI client appropriately.

message = response.content[0].text

# Validate response from LLM

In [6]:
print(message)

1. I tried to catch some fog, but I mist.
2. What do you call a fish wearing a bowtie? Sofishticated.
3. I heard the best time to go fishing is at the crack of dawn, so I couldn't help but wonder: How do you fix a broken dawn?
4. Fishing is reel fun, especially when you're angling for a good time!
5. I've been told I'm a good catch, but I think I'm more of a keeper.


In [7]:

print(response)


Message(id='msg_01WMuv5uMtony3DzQQmrmX5b', content=[ContentBlock(text="1. I tried to catch some fog, but I mist.\n2. What do you call a fish wearing a bowtie? Sofishticated.\n3. I heard the best time to go fishing is at the crack of dawn, so I couldn't help but wonder: How do you fix a broken dawn?\n4. Fishing is reel fun, especially when you're angling for a good time!\n5. I've been told I'm a good catch, but I think I'm more of a keeper.", type='text')], model='claude-3-opus-20240229', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(input_tokens=90, output_tokens=117))


# Check into Github
*   repository : "cooolbabu/GoogleGemini101"
*   filename : "AzureDatabricks/filename" - specify actual filename
*   filecontent: Contents of the file to check in
*   tag_name: give a comment. It will show in Github
* branch: branch name to check into. Ensure that branch already exists
          Future TODO: if branch doesn't exist (notify, ask, process)



In [8]:
repository = "cooolbabu/GoogleGemini101"
filename = "AzureDatabricks/claude_reponse_txt"
tag_name = "second iteration for comparision"
branch_name = "testing_claude"

check_in_file(repository, filename, message, tag_name, branch=branch_name)


File 'AzureDatabricks/claude_reponse_txt' updated successfully.


# Github Helpers

In [18]:
# @title Github Helper functions
repository = "cooolbabu/GoogleGemini101" # @param ["cooolbabu/GoogleGemini101"]


In [16]:
#pip install PyGithub
from google.colab import userdata
from github import Github

# Get the OpenAI API key from Colab secrets
github_token=userdata.get('Github_Token')
# Initialize a GitHub instance
g = Github(github_token)

def read_file_as_string(file_path):
  """
    Reads the file and return a string representation of the file contents

    Parameters:
      file_path (str): Filename including filepath
  """
  try:
      with open(file_path, 'r') as file:
          file_contents = file.read()
      return file_contents
  except FileNotFoundError:
      print(f"File '{file_path}' not found.")
      return None
  except Exception as e:
      print(f"An error occurred: {e}")
      return None

def check_in_file(repo_name, file_path, file_content, content_tag, branch):
  """
    Checks if a specific file exists in a GitHub repository and updates it with new content if it does.
    If the file does not exist, it creates a new file with the provided content.

    This function operates on a specific branch named 'test'. If updating, it will commit the changes with a given content tag as the commit message.
    In case the file needs to be created, it will also use the content tag as the commit message for the new file.

    Parameters:
    - repo_name (str): The name of the repository, formatted as 'username/repository'.
    - file_path (str): The path to the file within the repository. This should include the file name and its extension.
    - file_content (str): The content to be written to the file. This is used both for updating and creating the file.
    - content_tag (str): A message associated with the commit used for updating or creating the file.
    - branch (str): Github branch for the code

    Behavior:
    - If the file exists at the specified path, it updates the file with `file_content`, using `content_tag` as the commit message.
    - If the file does not exist, it creates a new file at the specified path with `file_content`, also using `content_tag` as the commit message for creation.
    - Upon successful update or creation, prints a success message indicating the action taken.
 """

  # Get the repository
  repo = g.get_repo(repo_name)

  try:
      # Get the contents of the file if it exists
      file = repo.get_contents(file_path, ref=branch)

      # Update the file
      repo.update_file(file_path, content_tag, file_content, file.sha, branch=branch)
      print(f"File '{file_path}' updated successfully.")
  except:
      # If the file doesn't exist, create it
      print(f"{file_path}/{file_content} does not exist")
      repo.create_file(file_path, content_tag, file_content, branch=branch)
      print(f"File '{file_path}' created successfully.")


In [ ]:
# @title Github functions

file_contents = read_file_as_string('/content/export_007.json')
check_in_file(repository, "AzureDatabricks/Test_file", file_contents, "Initial checkin of export_007_json", branch="test")
